In [ ]:
# import
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.metrics import confusion_matrix

import datetime
plt.rcParams["figure.figsize"] = (20,10)

In [62]:
# settings
shift = 15
window = 45
model_type = "autoreg15" # for autoreg - autoreg{shift]; for multiNN - PC, AsyH, ap_index, BzGSE, all_parameters
model_type2 = "autoreg" # for autoreg - autoreg; for multiNN - PC, AsyH, ap_index, BzGSE, all_parameters
best_model = True # load best model (true) or model from epoch (false)
epoch = "03"
mae = "0.0561"

In [63]:
# data loading
size=1 # select subset (>1) or whole test_data (1)
param_t = np.load(f"../data/{model_type2}/shift-{shift}-windows-{window}/test_index_timestamp.npy", allow_pickle = True).flatten()
param_ytest = np.load(f"../data/{model_type2}/shift-{shift}-windows-{window}/y_test.npy", allow_pickle = True).flatten()
param_bin = np.load(f"../data/{model_type2}/shift-{shift}-windows-{window}/test_index_bin.npy", allow_pickle = True).flatten()
if best_model:
    param_pred = np.load(f"../prediction/y_pred-{model_type2}-shift-{shift}-windows-{window}.npy, allow_pickle = True").flatten()
else:
    param_pred = np.load(f"../prediction/{model_type2}/shift-{shift}-windows-{window}/y_pred_epoch-{epoch}-mae-{mae}.npy", allow_pickle = True).flatten()

# create dataframe
param_df = pd.DataFrame(data={f"{model_type}_test": param_ytest[:int(len(param_ytest)/size)], f"{model_type}_pred": param_pred[:int(len(param_pred)/size)], f"{model_type}_bin":param_bin[:int(len(param_ytest)/size)], f"{model_type}_time":param_t[:int(len(param_ytest)/size)]})
param_df[f"{model_type}_bin"] = param_df[f"{model_type}_bin"].str[:5]

param_df["index_time_bin"]=param_df[f"{model_type}_time"]+" "+param_df[f"{model_type}_bin"]
param_df=param_df.set_index("index_time_bin")

#param_df.to_csv(f"{model_type}_predictions_on_test-data_with_time-bin.csv")


In [64]:
len(param_pred)

1010932

In [65]:
# load data (if you want load from .csv)
# data = pd.read_csv(f'{model_type}_predictions_on_test-data_with_time-bin.csv')

# or select data from param_df
data = param_df

data[f"{model_type}_time"] = pd.to_datetime(data[f"{model_type}_time"])
data

,autoreg15_test,autoreg15_pred,autoreg15_bin,autoreg15_time
index_time_bin,,,,
2019-01-01 01:27:00 14117,0.031,0.084112,14117,2019-01-01 01:27:00
2019-01-01 01:28:00 14117,0.029,0.084094,14117,2019-01-01 01:28:00
2019-01-01 01:29:00 14117,0.030,0.084053,14117,2019-01-01 01:29:00
2019-01-01 01:30:00 14117,0.033,0.084032,14117,2019-01-01 01:30:00
2019-01-01 01:31:00 14117,0.024,0.084053,14117,2019-01-01 01:31:00
...,...,...,...,...
2019-07-27 04:51:00 13828,0.027,0.084157,13828,2019-07-27 04:51:00
2019-07-27 04:52:00 13828,0.026,0.084144,13828,2019-07-27 04:52:00
2019-07-27 04:53:00 13828,0.027,0.084135,13828,2019-07-27 04:53:00


### Coefficient of variation

In [66]:
global_mean = data[f"{model_type}_test"].mean()
global_mean

0.03574626977877839

In [67]:
n = len(data)
pred_std=[]
test_std=[]
window_size_cov=[]

# create windows for CoV calculation
for i in range(n):
    subset = data[max(0,i-22):min(i+22+1, n)]
    subset = subset[subset[f"{model_type}_bin"]==data[f"{model_type}_bin"][i]]
    subset = subset[subset[f"{model_type}_time"] <= data[f"{model_type}_time"][i] + datetime.timedelta(minutes=22)]
    subset = subset[subset[f"{model_type}_time"] >= data[f"{model_type}_time"][i] - datetime.timedelta(minutes=22)]
    pred_std.append(subset[f"{model_type}_pred"].std())
    test_std.append(subset[f"{model_type}_test"].std())
    window_size_cov.append(len(subset))
    if i%10000 == 0: # control the speed of calculations
        print(i)

0
10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000
120000
130000
140000
150000
160000
170000
180000
190000
200000
210000
220000
230000
240000
250000
260000
270000
280000
290000
300000
310000
320000
330000
340000
350000
360000
370000
380000
390000
400000
410000
420000
430000
440000
450000
460000
470000
480000
490000
500000
510000
520000
530000
540000
550000
560000
570000
580000
590000
600000
610000
620000
630000
640000
650000
660000
670000
680000
690000
700000
710000
720000
730000
740000
750000
760000
770000
780000
790000
800000
810000
820000
830000
840000
850000
860000
870000
880000
890000
900000
910000
920000
930000
940000
950000
960000
970000
980000
990000
1000000
1010000


In [68]:
data[f"{model_type}_std_pred"] = pred_std
data[f"{model_type}_std_test"] = test_std

# calculate CoV
data[f"{model_type}_cov_pred"] = data[f"{model_type}_std_pred"]/global_mean
data[f"{model_type}_cov_test"] = data[f"{model_type}_std_test"]/global_mean

# if you want to save size of CoV windows
# data[f"{model_type}_count_for_cov"] = window_size_cov

In [25]:
# save data
data.to_csv(f'{model_type2}/{model_type}_predictions_on_test-data_with_time-bin_std-cov_full-test.csv')

In [26]:
data

,PC_test,PC_pred,PC_bin,PC_time,PC_std_pred,PC_std_test,PC_cov_pred,PC_cov_test,PC_count_for_cov
index_time_bin,,,,,,,,,
2019-01-01 01:27:00 14117,0.031,0.064558,14117,2019-01-01 01:27:00,0.005902,0.005525,0.165545,0.154981,16
2019-01-01 01:28:00 14117,0.029,0.064681,14117,2019-01-01 01:28:00,0.005771,0.005414,0.161862,0.151871,17
2019-01-01 01:29:00 14117,0.030,0.063228,14117,2019-01-01 01:29:00,0.005679,0.005347,0.159303,0.149973,18
2019-01-01 01:30:00 14117,0.033,0.062721,14117,2019-01-01 01:30:00,0.005577,0.005321,0.156428,0.149257,19
2019-01-01 01:31:00 14117,0.024,0.064875,14117,2019-01-01 01:31:00,0.005493,0.005194,0.154066,0.145684,20
...,...,...,...,...,...,...,...,...,...
2019-10-06 21:48:00 13927,0.029,0.062144,13927,2019-10-06 21:48:00,0.001003,0.001512,0.028124,0.042399,17
2019-10-06 21:49:00 13927,0.033,0.061490,13927,2019-10-06 21:49:00,0.001003,0.001512,0.028124,0.042399,17
2019-10-06 21:50:00 13927,0.031,0.061066,13927,2019-10-06 21:50:00,0.001003,0.001512,0.028124,0.042399,17
